# ELEC470 Equalizer Studies
- Modified HWs by HZF. Machine Learning methods are investigated
- Source : Rich Kozick, ELEC 470, Spring 1998
- TODO:
- (1) [-N, N] arası sistematik yapılsın
- (2) Plot spectrums

In [ ]:
#------------------------------------------------------------------------------#
# Import Libraries
#------------------------------------------------------------------------------#
import numpy as np
import matplotlib.pyplot as plt

pi = np.pi
from numpy import exp as exp
from numpy import log as log
from numpy import cos as cos
from numpy import sin as sin
from numpy import sqrt as sqrt
from numpy import sign as sign
from numpy import real as real
from numpy import imag as imag

from scipy import signal
from scipy.signal import convolve
from scipy.signal import lfilter
from scipy.linalg import toeplitz

import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
#------------------------------------------------------------------------------#
# Common functions. All inputs MUST be column vector. Otherwise undefined behaviour
#------------------------------------------------------------------------------#
def upsample_1d(x, N):
    # Create zero filled array
    r = np.zeros((x.reshape(-1,1).shape[0]*N,1))
    
    # Replace values and return
    r[::N] = x.reshape(-1,1)
    return r

#def eye_diagram(sig, dt, T):
#    plt.figure()
#    nT = int(T/dt) # Samples per symbol
#    plt.plot(np.arange(dt, 2*T+dt, dt), np.transpose(sig.reshape(-1, nT*2)))
#    return plt

def sig_ser(ref, recv):
    _ref = ref.reshape(-1)
    _recv = recv.reshape(-1)
    _s = _ref.size
    val = 0.0
    for k in range(0, _s):
        if (ref[k] != recv[k]):
            val += 1.0 
    return val/_s  

def sig_energy(sig):
    return np.sum(np.dot(sig.reshape(-1), sig.conj().reshape(-1)))
    
def sig_power(sig):
    return sig_energy(sig)/sig.size

def sig_evm_rms(ref, recv):
    _ref = ref.reshape(-1)
    _recv = recv.reshape(-1)
    return np.sqrt(sig_power(_ref-_recv)/sig_power(_ref))*100

def sig_fir(x, h, offset):
    _x = x.reshape(-1)
    _h = h.reshape(-1)
    _y = convolve(_h, _x).reshape(-1,1)
    start = offset
    end = _x.size + offset
    return _y[start:end]
    

## Simulating Communication Link

In [ ]:
#------------------------------------------------------------------------------#
# Data Parameters
#------------------------------------------------------------------------------#
T = 1 # Bit period
dt = 0.1 # Sampling time in simulation
nT = int(T/dt) # Samples per symbol
N = 1000 # Number of bits to generate
nx = N*nT # Total samples

#------------------------------------------------------------------------------#
# Channel Parameters
#------------------------------------------------------------------------------#
snr_db = 15
snr = 10**(snr_db/10)
tau = 3 # Time constant of channel
n_isi = 5*tau
nc = nT*(T + n_isi)
c = np.full( (nc,1), 0).reshape(-1,1).astype(float)

#------------------------------------------------------------------------------#
# Equalizer Parameters
#------------------------------------------------------------------------------#
Ne = int(n_isi/T) # Number of equalizer samples

In [ ]:
#------------------------------------------------------------------------------#
# Create output pulse: rectangular pulse convolved with first-order low-pass 
# filter impulse response
#------------------------------------------------------------------------------#
# Pulse in symbol period
t1 = np.arange(dt, T+dt, dt).reshape(-1,1)
c[0:nT] = 1 - exp(-t1/tau)

# Interfered part
t2 = np.arange(T+dt, T+n_isi+dt, dt).reshape(-1,1)
c[nT:nT+t2.shape[0]] = c[nT-1]*exp(-(t2-T)/tau)


# Get samples of c(t) to solve for equalizer weights
cT = c[nT-1:nc+nT:nT]

# Plotting
plt.figure()
#plt.subplot(1,2,1)
plt.title('Pulse Shaping')
plt.plot(np.concatenate([t1,t2]), c)
plt.stem(np.arange(nT, nc+nT, nT)*dt, cT, basefmt=" ", linefmt="red", markerfmt="ro")
plt.grid()
plt.xlabel('Time (sec)')
plt.ylabel('c(t)')
plt.legend(['Smeared pulse c(t)', 'Sampled c[nT]'])
plt.show()


In [ ]:
#------------------------------------------------------------------------------#
# Generate bit stream
#------------------------------------------------------------------------------#
bits = np.random.normal(0, 1, N)>0
bits = bits.reshape(-1,1)
symbols = 2*bits - 1;

# Plotting
plt.figure()
plt.title('Bit stream values b[n]')
plt.stem(bits, basefmt=" ")
plt.grid()
plt.xlabel('Bits')
plt.ylabel('b[n]')
plt.show()


In [ ]:
#------------------------------------------------------------------------------#
# Received signal with ISI
#------------------------------------------------------------------------------#
x = upsample_1d(symbols, nT)
x = lfilter(c.reshape(-1), 1, x.reshape(-1)).reshape(-1,1)
#x = convolve(c.reshape(-1), x.reshape(-1)).reshape(-1,1)
#x = x[:x.shape[0]-c.shape[0]+1]

# Add noise of the specified level
s_p = sig_power(x) # Signal power
n_p = s_p/snr # Noise power
noise = sqrt(n_p) * np.random.normal(0, 1, x.size).reshape(-1,1)
x = x + noise;
xT = x[nT-1::nT]

# Plot received signal
plt.figure()
plt.plot(np.arange(0, 2*T, dt), np.transpose(x.reshape(-1, nT*2)))
plt.title('Eye diagram of received signal')
plt.grid()
plt.xlabel('Time')
plt.ylabel('x')
plt.show()

plt.figure()
plt.scatter(real(xT), imag(xT))
plt.title('Constellation of received signal')
plt.grid()
plt.xlabel('In-phase')
plt.ylabel('Quadrature')
plt.show()

## Conventional Equalizers

In [ ]:
#------------------------------------------------------------------------------#
# Common Equalization
#------------------------------------------------------------------------------#
# Toeplitz convolution matrix of the channel on the left side of ZF equalizer equation
padding = np.zeros((Ne,1), cT.dtype)
first_col = np.r_[cT, padding]
first_row = np.r_[[cT[0]], padding, padding]
C = toeplitz(first_col, first_row)

In [ ]:
#------------------------------------------------------------------------------#
# ZF Equalization
#------------------------------------------------------------------------------#
# Right side of ZF equalizer weight equation
r = np.r_[padding, [[1]], padding]

# Solve for ZF equalizer weights
w_zf, resid, rank, s = np.linalg.lstsq(C, r, rcond=-1) # Left matrix division, rcond is about internals of python
h_zf = upsample_1d(w_zf, nT)

# Filter signal
y_zf = sig_fir(x, h_zf, nT*Ne)
yT_zf = y_zf[nT-1::nT]
yT_zf_syms = np.sign(yT_zf)
print("ZF Receiver:\n\tSER: " + str(sig_ser(symbols, yT_zf_syms)) + "\n\tEvm  :" + str(sig_evm_rms(symbols, yT_zf)))

# Plot ZF equalized signal
plt.figure()
plt.plot(np.arange(dt, 2*T+dt, dt), np.transpose(y_zf.reshape(-1, nT*2)))
plt.title('Eye diagram of received signal')
plt.grid()
plt.xlabel('Time')
plt.ylabel('x')
plt.show()

plt.figure()
plt.scatter(real(yT_zf), imag(yT_zf))
plt.title('Constellation of received signal')
plt.grid()
plt.xlabel('In-phase')
plt.ylabel('Quadrature')
plt.show()


In [ ]:
#------------------------------------------------------------------------------#
# Wiener Equalization
#------------------------------------------------------------------------------#
Rxx = np.zeros((2*Ne+1,2*Ne+1))
Rxd = np.zeros((2*Ne+1,1))
for k in range(Ne,N-Ne):
    xk = xT[(k-Ne):(k+Ne+1)]
    xk_r = np.flipud(xk)    
    Rxx = Rxx + np.dot(xk_r, xk_r.conj().T)
    Rxd = Rxd + symbols[k]*xk_r

# Obtain wiener filter coefficients
w_wiener, resid, rank, s = np.linalg.lstsq(Rxx, Rxd, rcond=-1) # Left matrix division, rcond is about internals of python
h_wiener = upsample_1d(w_wiener, nT)

# Filter signal
y_wiener = sig_fir(x, h_wiener, nT*Ne)
yT_wiener = y_wiener[nT-1::nT] # Sampling
yT_wiener_syms = np.sign(yT_wiener) # Symbol estimation
print("Wiener Receiver:\n\tSER: " + str(sig_ser(symbols, yT_wiener_syms)) + "\n\tEvm  :" + str(sig_evm_rms(symbols, yT_wiener)))

# Plot Wiener equalized signal
plt.figure()
plt.plot(np.arange(dt, 2*T + dt, dt), np.transpose(y_wiener.reshape(-1, nT*2)))
plt.title('Eye diagram of received signal')
plt.grid()
plt.xlabel('Time')
plt.ylabel('x')
plt.show()

plt.figure()
plt.scatter(real(yT_wiener), imag(yT_wiener))
plt.title('Constellation of received signal')
plt.grid()
plt.xlabel('In-phase')
plt.ylabel('Quadrature')
plt.show()

In [ ]:
#------------------------------------------------------------------------------#
# Adaptive equalizer with LMS algorithm
#------------------------------------------------------------------------------#
# Memory allocation to record evolution of LMS
wrec = np.zeros((2*Ne+1, N-2*Ne))
evm_lms = np.zeros(N-2*Ne)

# LMS parameters and initial weights
mu_lms = 0.04
w_lms = wrec[:,0].reshape(-1, 1) # Initialize weights to zero

# LMS algorithm to estimate the equalizer weights in real-time using the training data
for k in range(Ne,N-Ne):
    xk = xT[(k-Ne):(k+Ne+1)]
    xk_r = np.flipud(xk).reshape(-1,1)
    yk = np.dot(w_lms.T,xk_r)
    ek = symbols[k] - yk
    w_lms = w_lms + mu_lms*ek*xk_r
    wrec[:, k-Ne] = w_lms.flatten()
    
    # Equalize for each weight to monitor EVM
    yT_k = sig_fir(xT, w_lms, Ne)
    evm_lms[k-Ne] = sig_evm_rms(symbols, yT_k)

yT_lms = sig_fir(xT, w_lms, Ne)
yT_lms_syms = np.sign(yT_k) # Symbol estimation
print("LMS Equalizer:\n\tSER: " + str(sig_ser(symbols, yT_lms_syms)) + "\n\tEvm  :" + str(sig_evm_rms(symbols, yT_lms)))

# Plot Wiener equalized signal
plt.figure()
plt.plot(evm_lms)
plt.grid()
plt.title('LMS Equalizer with ' + str(N) + ' training bits and mu: ' + str(mu_lms))
plt.xlabel('Training sample number')
plt.ylabel('EVM')

plt.figure()
plt.plot(wrec.T)
plt.grid()
plt.title('Evolution of LMS equalizer weights')
plt.xlabel('Training sample number')
plt.ylabel('Weight value')
plt.show()                      

## Machine Learning Methods
WARNING: Following codes are hobby-level. Don't take them serious

In [ ]:
#------------------------------------------------------------------------------#
# Machine Learning
#------------------------------------------------------------------------------#
# Allocate mems
x_set = np.zeros((1+Ne,N-2*Ne))
y_set = np.zeros((N-2*Ne,1))

# Get x and y set
for k in range(Ne,N-Ne):
    x_set[:, k-Ne] = xk = xT[(k-Ne):(k+1)].flatten()
    y_set[k-Ne] = symbols[k]
    
x_set = x_set.T
    
# Creating pandas dataframe from numpy array
dataset = pd.DataFrame({'x[M]': x_set[:, 0], 'x[M+1]': x_set[:, 1],'x[M+2]': x_set[:, 2], 'x[M+3]': x_set[:, 3],'x[M+4]': x_set[:, 4], 'x[M+5]': x_set[:, 5],'x[M+6]': x_set[:, 6], 'x[M+7]': x_set[:, 7], 'x[M+8]': x_set[:, 8], 'x[M+9]': x_set[:, 9],'x[M+10]': x_set[:, 10], 'x[M+11]': x_set[:, 11],'x[M+12]': x_set[:, 12], 'x[M+13]': x_set[:, 13],'x[M+14]': x_set[:, 14], 'x[M+15]': x_set[:, 15]})
dataset['y'] = y_set

dataset.head()

In [ ]:
# Form train and test set
x_train, x_test, y_train, y_test = train_test_split(dataset.iloc[:,:16], dataset['y'], test_size=0.33, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Random Forest Classifier
model = RandomForestClassifier()
model.fit(x_train, y_train)

# Predict from model
y_pred = model.predict(x_test)

# Results
print('SNR: ' + str(snr_db) + ' [db]\n\tSER: ' + str(1-accuracy_score(y_test, y_pred)))
print('\nClassification Report:')
print(classification_report(y_test, y_pred))

In [ ]:
# LSTM
model_lstm = Sequential()

# Alternative-1
model_lstm.add(LSTM(1, batch_input_shape=(None, Ne+1, 1), return_sequences=False))

#Alternative-2
#model_lstm.add(LSTM(1, batch_input_shape=(None, Ne+1, 1), return_sequences=True))
#model_lstm.add(LSTM(1, return_sequences=False))

model_lstm.compile(loss='mean_absolute_error', optimizer='adam',metrics=['accuracy'])
model_lstm.summary()

In [ ]:
# Reshape to 3d for ltsm
_x_train = x_set[0:650, :]
_x_test = x_set[650:x_set.shape[0]+1, :]

_y_train = y_set[0:650,]
_y_test = y_set[650:y_set.shape[0]+1]

_x_train = _x_train.reshape(_x_train.shape[0], _x_train.shape[1], 1)
_y_train = _y_train.reshape(_y_train.shape[0], _y_train.shape[1], 1)

_x_test = _x_test.reshape(_x_test.shape[0], _x_test.shape[1], 1)
_y_test = _y_test.reshape(_y_test.shape[0], _y_test.shape[1], 1)

In [ ]:
# Train model
history_ltsm = model_lstm.fit(_x_train, _y_train, epochs=400, validation_data=(_x_test, _y_test))

In [ ]:
results_lstm = model_lstm.predict(_x_test)

len = 25 #_y_test.shape[0]
plt.scatter(range(len), results_lstm[0:len].reshape(len), c='r')
plt.scatter(range(len), _y_test[0:len].reshape(len), c='g')

In [ ]:
plt.plot(history_ltsm.history['loss'])
plt.show()